In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
student_data = pd.read_excel("Raw Data/Anon Enrollment Data.xlsx")
enrol_nums_data = pd.read_excel("Raw Data/Course Enrollment Numbers.xlsx", sheet_name="Course Numbers")
enrol_nums_additional = pd.read_excel("Raw Data/Course Enrollment Numbers.xlsx", sheet_name="abbreviated course names")
timetable_data = pd.read_excel("Raw Data/School of Mathematics - Timetable Data.xlsx")
room_data = pd.read_excel("Raw Data/Timetabling KB Rooms_5205_Central.xlsx")

In [3]:
unique_students = student_data["UUN"].unique()
unique_years = student_data["Year Of Programme"].unique()
unique_degrees = student_data["Programme Of Study Sought Title"].unique()
unique_schools = student_data["Programme School Name"].unique()
unique_years_taken = student_data["Normal Year Taken"].unique()
unique_course_codes = student_data['Course Code'].unique()
unique_courses = student_data['Course Name'].unique()

In [4]:
enrol_nums_cols = list(enrol_nums_data.columns)

In [5]:
student_cols = list(student_data.columns)

In [6]:
math_courses_only = student_data[student_data["Course Code"].str.startswith("MATH")]
math_courses = math_courses_only["Course Name"].unique()

enrol_nums_courses = list(enrol_nums_data["Course"])
uncommon_maths_courses = [i for i in math_courses if i not in enrol_nums_courses]

In [7]:
# uncommon_maths_courses

In [8]:
with open('Keys/course_abbreviation_key.txt', 'r') as file:
    # Read the entire contents of the file
    abbr_key = file.read()
file.close()

abbr_key = abbr_key.split("\n")
abbr_key = [i.strip() for i in abbr_key if i.strip()]

In [9]:
enrol_nums_copy = enrol_nums_data.copy()

In [10]:
abbr_map = {}
for course in abbr_key:
    abbreviation, description, code = course.split(' = ')
    abbr_map[code] = abbreviation

enrol_nums_copy["Abbreviated Course Name"] = enrol_nums_copy["Code"].map(abbr_map)

In [11]:
len(enrol_nums_copy["Abbreviated Course Name"]) == len(enrol_nums_additional["Abbreviated Course Name"])

True

In [12]:
enrol_nums_copy["Abbreviated Course Name"].all() == enrol_nums_additional["Abbreviated Course Name"].all()

True

# Changing Room Data

In [13]:
room_copy = room_data.copy()
new_room_cols = ["campus", "building", "room_name", "capacity",
                 "ownership", "gt_or_ts", "school_priority", "room_layout","furniture_config",
                 "control_system", "desktop", "induction_system", "lec_recording", "microphone", "pres_facilities", 
                 "sound_system", "wall_mounted_writing"]

room_copy.columns = new_room_cols

In [14]:
unique_campuses = room_copy["campus"].unique()
unique_buildings = room_copy["building"].unique()
unique_rooms = room_copy["room_name"].unique()
unique_ownership = room_copy["ownership"].unique()

In [15]:
room_copy["campus"] = room_copy["campus"].replace({"King's Buildings Campus":"KB", 
                                                   "Central":"central"})


# building_replacements = {"Alrick":"ALR",
#                          "Ashworth":"ASH",
#                          "Daniel Rutherford Building":"DRB",
#                          "Eng Sanderson Building":"ESB",
#                          "Grant Institute":"GRA",
#                          "Hudson Beare Building":"HBB",
#                          "Joseph Black Building":"JBB",
#                          "JCMB":"JCMB",
#                          "Murchison House":"MH",
#                          "Nucleus":"NUC",
#                          "Swann Building":"SB",
#                          "Gordon Aikman Lecture Theatre":"GALT",
#                          "Appleton Tower":"AT",
#                          "40 George Square":"40GS",
#                          "Chrystal MacMillan Building":"CMC",
#                          "Old College":"OC",
#                          "Lister Learning and Teaching Centre":"LLTC",
#                          "Robson Building":"ROB"}

# room_copy["building"] = room_copy["building"].replace(building_replacements)


allocation_replacements = {"1. Centrally Allocated Space":"centrally_allocated", "1. Locally Allocated Space": "locally_allocated"}
room_copy["ownership"] = room_copy["ownership"].replace(allocation_replacements)


gt_ts_replacements = {"2. General Teaching":"GT", "2. Teaching Studio":"TS"}
room_copy["gt_or_ts"] = room_copy["gt_or_ts"].replace(gt_ts_replacements)
room_copy.loc[room_copy["room_name"].str.contains("Rowan"), "gt_or_ts"] = "TS"

In [16]:
room_copy["school_priority"] = room_copy["school_priority"].str.split(" - ").str[1]
room_copy["room_layout"] = room_copy["room_layout"].str.split(" - ").str[1]

In [17]:
room_copy["school_priority"] = room_copy["school_priority"].fillna("no priority")

priority_replacements = {"Engineering":"eng",
                         "Biological":"bio",
                         "Geosciences":"geo",
                         "Chemistry":"chem",
                         "Mathematics":"maths",
                         "Physics and Astronomy":"phys",
                         "Mathematics/ Physics and Astronomy":"maths_phys"}
room_copy["school_priority"] = room_copy["school_priority"].replace(priority_replacements)


layout_replacements = {"Classroom Style":"classroom",
                       "Theatre Style":"theatre",
                       "Boardroom Style":"boardroom", 
                       "Computer Lab Style":"comp_lab"}
room_copy["room_layout"] = room_copy["room_layout"].replace(layout_replacements)
room_copy.loc[room_copy["room_name"].str.contains("Teaching Studio"), "room_layout"] = "classroom"
room_copy.loc[room_copy["room_name"].str.contains("Rowan"), "room_layout"] = "classroom"
room_copy.loc[room_copy["room_name"].str.contains("JCMB_6301"), "room_layout"] = "classroom"

In [18]:
room_copy["furniture_config"] = room_copy["furniture_config"].str.split("\. ").str[1]

In [19]:
furniture_replacements = {"Tables and Chairs":"tables_chairs",
                          "Flexible seating (not tables)":"flex_seating",
                          "Retractable Seating":"retract_seating"}
room_copy["furniture_config"] = room_copy["furniture_config"].replace(furniture_replacements)

In [20]:
list(room_copy["furniture_config"].unique())

['tables_chairs', 'flex_seating', nan, 'retract_seating']

In [21]:
true_false_cols = ["control_system", "desktop", "induction_system", "lec_recording", "microphone", "pres_facilities", "sound_system", "wall_mounted_writing"]

for col in true_false_cols:
    room_copy[col] = room_copy[col].apply(lambda x: True if pd.notna(x) and x.strip() else False)

In [22]:
dummy_enc_cols = []

room_furniture_encoded = pd.get_dummies(room_copy["furniture_config"], prefix="furniture")
room_layout_encoded = pd.get_dummies(room_copy["room_layout"], prefix="layout")
gt_ts_encoded = pd.get_dummies(room_copy["gt_or_ts"])
ownership_encoded = pd.get_dummies(room_copy["ownership"])
priority_encoded = pd.get_dummies(room_copy["school_priority"], prefix="priority")


room_copy = pd.concat([room_copy, room_furniture_encoded, room_layout_encoded, gt_ts_encoded, ownership_encoded, priority_encoded], axis=1)
# to_drop = ["furniture_config", "room_layout", "gt_or_ts", "ownership", "school_priority"]
# room_copy = room_copy.drop(to_drop, axis=1)

In [23]:
encoded_room_data = room_copy

encoded_room_data.to_excel("Processed Data/encoded_room_data.xlsx", index=False)

# Cleaning Timetabling Data

In [46]:
timetable_copy = timetable_data.copy()
timetable_copy = timetable_data.rename(columns = {"Course Name":"Course", 
                                                  "Course Code":"Code",
                                                  "Allocated Location Name": "room_name"})

timetable_copy = pd.merge(timetable_copy, enrol_nums_additional[["Delivery Period", "Normal Year Taken", "Code"]], on = "Code", how = "left")
timetable_copy = pd.merge(timetable_copy, room_copy[["building", "room_name"]], on = "room_name", how = "left")
timetable_copy.shape[0] == timetable_data.shape[0]

True

In [47]:
day_key = {}
with open("Keys/day_key.txt") as file:
    for line in file:
        (day, val) = line.split(" = ")
        day_key[str(day)] = int(val)
   
timetable_copy["encoded_days"] = timetable_copy["Scheduled Days"].map(day_key)
# timetable_copy = timetable_copy.infer_objects(copy=False) # this just removes a weird future warning

In [49]:
timetable_copy.columns

Index(['Course', 'Code', 'Linked Course', 'Activity', 'Activity Type Name',
       'Planned Size', 'Real Size', 'Delivery Semester',
       'Teaching Week Pattern', 'Number Of Teaching Weeks', 'Scheduled Days',
       'Scheduled Start Time', 'Scheduled End Time', 'Duration', 'Total Hours',
       'Zone Name', 'room_name', 'Delivery Period', 'Normal Year Taken',
       'building', 'encoded_days'],
      dtype='object')

In [50]:
uncommon_rooms = list(timetable_copy[timetable_copy["building"].isnull()]["room_name"].unique())

In [51]:
timetable_copy["Abbreviated Course Name"] = timetable_copy["Code"].map(abbr_map)

def split_on_star(entry):
    if isinstance(entry, str) and '*' in entry:
        return entry.split('*')[1]
    else:
        return entry

cols_w_star = ["Activity Type Name", "Delivery Semester", "Zone Name"]

for col in cols_w_star:
    timetable_copy[col] = timetable_copy[col].apply(split_on_star)

In [52]:
def num_list(entry):
    numbers = []
    for part in entry.split(','):
        if '-' in part:
            start, end = map(int, part.split('-'))
            numbers.extend(range(start, end + 1))
        else:
            numbers.append(int(part))
    return numbers

timetable_copy["Teaching Week Pattern"] = timetable_copy["Teaching Week Pattern"].apply(num_list)


week_key = {}
with open("Keys/week_codes.txt", 'r', encoding='utf-8') as file:
    for line in file:
        line = line.replace('\u2060', '')
        
        line = line.strip()
        if line:
            key_part = line.split('=')
            key_part[0] = int(key_part[0].strip())
            week_key[key_part[0]] = key_part[1].strip()

def convert_to_strings(numbers):
    return [week_key[number] for number in numbers]

timetable_copy["Teaching Week Pattern"] = timetable_copy["Teaching Week Pattern"].apply(convert_to_strings)
timetable_copy["Teaching Week Pattern"] = timetable_copy["Teaching Week Pattern"].apply(lambda x: ", ".join(x))

In [53]:
time_periods = {}
with open('Keys/time_periods.txt', 'r') as file:
    for line in file:
        period, times = line.strip().split('=')
        start, end = times[1:].split(', ')
        time_periods[int(period.strip())] = [start.strip("["), end.strip("]")]
# time_periods

In [54]:
def get_time_periods(start, end):
    encompassed_periods = []
    for period, (period_start, period_end) in time_periods.items():
        start_hour, start_minute = map(int, period_start.split(':'))
        end_hour, end_minute = map(int, period_end.split(':'))
        
        class_start_hour, class_start_minute = map(int, start.split(':'))
        class_end_hour, class_end_minute = map(int, end.split(':'))
        
        # Compare hours and minutes
        if (class_start_hour == start_hour and class_start_minute == start_minute) and (class_end_hour == end_hour and class_end_minute == end_minute):
            encompassed_periods.append(period)
        elif (class_start_hour < end_hour or (class_start_hour == end_hour and class_start_minute < end_minute)) and \
           (class_end_hour > start_hour or (class_end_hour == start_hour and class_end_minute > start_minute)):
            encompassed_periods.append(period)
    return encompassed_periods

# Apply function to each row and create new column
timetable_copy['time_periods'] = timetable_copy.apply(lambda x: get_time_periods(x['Scheduled Start Time'], x['Scheduled End Time']), axis=1)
timetable_copy["time_periods"] = timetable_copy["time_periods"].apply(lambda x: ", ".join(map(str,x)))

In [55]:
hm_duration = [map(int, time.split(':')) for time in timetable_copy["Duration"]]
nhours_duration = [(h + m / 60) for h, m in hm_duration]
timetable_copy["duration_hours"] = nhours_duration

hm_total = [map(int, time.split(':')) for time in timetable_copy["Total Hours"]]
nhours_total = [(h + m / 60) for h, m in hm_total]
timetable_copy["total_duration_hours"] = nhours_total

In [56]:
class_type_map = {}
with open("Keys/class_type_key.txt", "r") as file:
    for line in file:
        type_orig, type_abbr = line.split(" = ")
        class_type_map[type_orig] = type_abbr.strip()

timetable_copy["course_classtype"] = timetable_copy["Code"] + "_" + timetable_copy["Activity Type Name"].map(class_type_map)

In [57]:
def clean_ws_names(class_info):
    if "<" in class_info and ">" in class_info:
        return class_info.split("<")[0]
    else: 
        return class_info
    
timetable_copy["Activity"] = timetable_copy ["Activity"].apply(clean_ws_names)

def extract_ws_group(class_info):
    if "Workshop" in class_info and "/" in class_info:
        return class_info.split("/")[-1].strip()
    elif "Workshop" in class_info and class_info.split(" - ")[-1].strip() == "Workshop":
        return "00"
    elif "Workshop" in class_info:
        return "00"
    else:
        return np.nan
    
def clean_brackets(group):
    if "(reserve)" in str(group) or "(reserved)" in str(group):
        group = group.replace("(", "").replace(")", "").replace("d", "")
        group = group.replace(" ", "_")
        return group
    else:
        return group
    
timetable_copy["ws_groups"] = timetable_copy["course_classtype"] + "_" + timetable_copy["Activity"].apply(extract_ws_group)
timetable_copy["ws_groups"] = timetable_copy["ws_groups"].apply(clean_brackets)

In [58]:
unique_classes = timetable_copy.groupby(["Teaching Week Pattern", 
                                         "Scheduled Days", 
                                         "time_periods", 
                                         "course_classtype"]).size().reset_index()

# Custom function to aggregate ws_groups for each unique combination
def aggregate_ws_groups(combination):
    # Filter the DataFrame based on the current combination of columns
    filtered_df = timetable_copy[
        (timetable_copy["Teaching Week Pattern"] == combination["Teaching Week Pattern"]) &
        (timetable_copy["Scheduled Days"] == combination["Scheduled Days"]) &
        (timetable_copy["time_periods"] == combination["time_periods"]) &
        (timetable_copy["course_classtype"] == combination["course_classtype"])
    ]
    # Aggregate ws_groups for the filtered DataFrame
    aggregated_ws_groups = set(filtered_df["ws_groups"])
    return aggregated_ws_groups

# Apply the custom function to each unique combination
unique_classes["aggregated_ws_groups"] = unique_classes.apply(aggregate_ws_groups, axis=1)


timetable_copy = pd.merge(timetable_copy, unique_classes, on=['Teaching Week Pattern', 
                                                              'Scheduled Days', 
                                                              'time_periods', 
                                                              'course_classtype'], how='left')

def update_ws_groups(row):
    aggregated_ws_groups = row['aggregated_ws_groups']
    
    if pd.isna(aggregated_ws_groups):
        aggregated_ws_groups = np.nan
    else:
        aggregated_ws_groups = [aggregated_ws_groups]
    
    return aggregated_ws_groups

timetable_copy['aggregated_ws_groups'] = timetable_copy.apply(update_ws_groups, axis=1)
timetable_copy["aggregated_ws_groups"] = timetable_copy["aggregated_ws_groups"].apply(lambda x: ", ".join(map(str,x)))

In [59]:
def extract_elements(text):
    return re.findall(r"'([^']*)'", text)

# Apply the function to the column
timetable_copy['aggregated_ws_groups'] = timetable_copy['aggregated_ws_groups'].apply(extract_elements)
timetable_copy["aggregated_ws_groups"] = timetable_copy["aggregated_ws_groups"].apply(lambda x: ", ".join(map(str,x)))

timetable_copy["aggregated_ws_groups"] = timetable_copy["aggregated_ws_groups"].mask(timetable_copy["aggregated_ws_groups"] == '', np.nan)


timetable_copy.rename(columns = {0 : "no_of_agg_ws_groups"}, inplace=True)

In [60]:
cleaned_wsg_agg = timetable_copy["ws_groups"].dropna()

wsg_agg_dups = list(cleaned_wsg_agg[cleaned_wsg_agg.duplicated(keep=False)])

set([wsg_agg_dups[i][:9] for i in range(len(wsg_agg_dups))])

{'MATH08074',
 'MATH10010',
 'MATH10024',
 'MATH10067',
 'MATH10068',
 'MATH10079',
 'MATH10099',
 'MATH11148',
 'MATH11197'}

### Need to use the `Teaching Week Pattern` column to add in data for the `Delivery Semester` column.

In [61]:
unique_patterns = list(timetable_copy["Teaching Week Pattern"].unique())

sem1_class_patterns = [pattern for pattern in unique_patterns if "s1" in str(pattern)]
sem2_class_patterns = [pattern for pattern in unique_patterns if "s2" in str(pattern)]

In [62]:
# sorted(unique_patterns, key=len)

In [63]:
# sorted(sem2_class_patterns, key = len)

In [64]:
# Function to extract unique elements from all lists in the column
def extract_unique_elements(lst):
    unique_elements = set()
    for item in lst:
        unique_elements.update(item.split(', '))

    def sort_week(week):
        if "s1w" in str(week):
            numeric_part = int(week.split("s1w")[1])
        elif "s2w" in str(week):
            numeric_part = int(week.split("s2w")[1])
        else:
            return np.nan
        return numeric_part
    
    # Sort the unique elements based on the custom sorting function
    sorted_unique_elements = sorted(list(unique_elements), key=sort_week)
    return sorted_unique_elements

# Extract unique elements from all lists in the column
unique_elements_sem1 = extract_unique_elements(sem1_class_patterns)
unique_elements_sem2 = extract_unique_elements(sem2_class_patterns)

# Function to generate 0's and 1's list based on the presence of elements
def generate_binary_list(lst, unique_elements):
    binary_list = []
    for element in unique_elements:
        if element in lst:
            binary_list.append(1)
        else:
            binary_list.append(0)
    return binary_list

# Apply the function to each row in the column to create the new column
timetable_copy["sem1_pattern_one_hot"] = timetable_copy["Teaching Week Pattern"].apply(lambda x: generate_binary_list(x.split(", "), unique_elements_sem1))
timetable_copy["sem1_pattern_one_hot"] = timetable_copy["sem1_pattern_one_hot"].apply(lambda x: ", ".join(map(str,x)))

timetable_copy["sem2_pattern_one_hot"] = timetable_copy["Teaching Week Pattern"].apply(lambda x: generate_binary_list(x.split(", "), unique_elements_sem2))
timetable_copy["sem2_pattern_one_hot"] = timetable_copy["sem2_pattern_one_hot"].apply(lambda x: ", ".join(map(str,x)))

In [65]:
timetable_copy["room_name"] = timetable_copy["room_name"].str.split(',')
timetable_copy = timetable_copy.explode("room_name")
timetable_copy.reset_index(drop=True, inplace=True)

In [66]:
timetable_copy.loc[(timetable_copy['Code'] == 'MATH10069') & (timetable_copy['Activity'].str.contains('Skills Computer Workshop')), 'Activity Type Name'] = 'Workshop'

In [67]:
timetable_copy.to_excel("Processed Data/encoded_timetable_data.xlsx", index = False)

# Testing

In [68]:
list(timetable_copy.columns)

['Course',
 'Code',
 'Linked Course',
 'Activity',
 'Activity Type Name',
 'Planned Size',
 'Real Size',
 'Delivery Semester',
 'Teaching Week Pattern',
 'Number Of Teaching Weeks',
 'Scheduled Days',
 'Scheduled Start Time',
 'Scheduled End Time',
 'Duration',
 'Total Hours',
 'Zone Name',
 'room_name',
 'Delivery Period',
 'Normal Year Taken',
 'building',
 'encoded_days',
 'Abbreviated Course Name',
 'time_periods',
 'duration_hours',
 'total_duration_hours',
 'course_classtype',
 'ws_groups',
 'no_of_agg_ws_groups',
 'aggregated_ws_groups',
 'sem1_pattern_one_hot',
 'sem2_pattern_one_hot']